# Text als Muster

Die in Dokumentenformaten wie HTML definierte Struktur hilft uns nur soweit, wie die relevanten Inhalte eines Textes auch explizit gekennzeichnet sind. Oft sind aber Informationen in Texten in unstrukturierter Form enthalten. Diese Informationen in strukturierter Form zu extrahieren ist die Aufgabe der [Informationsextraktion](http://de.wikipedia.org/wiki/Informationsextraktion).

Im Falle der politischen Reden etwa sind die Datumsangaben in natürlicher Sprache verfasst, also z.B. »27. Januar 2011«. Dies ist zunächst einfach eine Zeichenkette (string) aus Zahlzeichen, Buchstaben, Leerzeichen und Punkten. Mit dieser Angabe kann man etwa nicht ohne weitere Verarbeitung bestimmen, ob eine Rede im Jahr 2011 oder im Jahr 2012 gehalten wurde. Dazu muss die unstrukturierte Zeichenkette in bestimmte Elemente, etwa Tag, Monat und Jahr, zerlegt werden.

In Python gibt es den Datentyp `date`, der den strukturierten Umgang mit Datumsangaben erlaubt:

In [1]:
import datetime
datetime.date.today()

datetime.date(2014, 10, 7)

In [2]:
date = datetime.date(2011, 1, 27)
date.year == 2011

True

Die Aufgabe besteht also zunächst darin, aus der Zeichenkette »27. Januar 2011« das Datum in strukturierter Form zu extrahieren. Dabei hilft es, dass wir wissen, dass hier zunächst der Tag steht, dann der Monat und dann das Jahr. Mit dieser Information lässt sich etwa das Jahr extrahieren.

In [3]:
date_string = '27. Januar 2011'
day, month, year = date_string.split()
int(year) == 2011

True

Für den Monat ist dies schwieriger, da er nicht als Zahl, sondern als Wort angegeben ist. In Python sind die hierfür notwendigen Informationen schon enthalten. Um aus einer Zeichenkette, die einem bestimmten Aufbau folgt, ein Datum zu extrahieren, muss man nur das entsprechende Muster formulieren:

> Erst der Tag als Zahl, gefolgt von einem Punkt, dann der Monat als ausgeschriebenes Wort, dann das vierstellige Jahr.

Die entsprechenden Muster lassen sich in der [Python-Dokumentation](https://docs.python.org/2.7/library/datetime.html?highlight=strptime#strftime-strptime-behavior) oder auf [strftime.org](http://strftime.org/) nachlesen. Das Muster lautet dann:

In [4]:
pattern = '%d. %B %Y'

Die Funktion `strptime` (string parse time) erlaubt, dieses Muster für die Extraktion eines Datums anzuwenden. Dabei wird standardmäßig ein `datetime`-Objekt erstellt, dass nicht nur da Datum, sondern auch eine Zeitangabe enthält. Da dies hier nicht benötigt wird, lässt sich daraus aber auch nur das Datum extrahieren. Da Python zusätzlich wissen muss, dass die Monate auf Deutsch angegeben sind, muss die entsprechende Sprache aktiviert werden.

In [5]:
import locale
locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')

date_and_time = datetime.datetime.strptime(date_string, pattern)
date_and_time.date()

datetime.date(2011, 1, 27)

Diese Art der Mustererkennung ist allerdings auf Datumsangaben beschränkt. Auf ähnliche Art und Weise kann man aber in vielen Fällen Muster in Texten extrahieren. Hierfür unterstützen die meisten Programmiersprachen sogenannte »reguläre Ausdrücke«. Diese erlauben, Muster zu beschreiben und in Texten zu finden. Ein regulärer Ausdruck besteht in der Regel aus diesen Elementen:

*Was?*

Welche Zeichen sollen gefunden werden?

* Wörtliche Zeichen: Zeichen werden so gefunden, wie sie eingegeben werden, z.B. »`a`«.
* Zeichen aus einer bestimmten Menge, z.B. »`[aä]`« für a oder ä, »`[a-z]`« für die Kleinbuchstaben zwischen a und z.
* Zeichen aus einer vordefinierten Menge, z.B. »`\d`« für Zahlen *(decimal)* oder »`\w`« für Buchstaben und Zahlen *(word)*.
* Ein beliebiges Zeichen: »`.`«.

*Wie viel?*

Wie viele dieser Zeichen sollen gefunden werden?

* »`*`«: Keinmal oder mehrfach
* »`+`«: Einmal oder mehrfach
* »`?`«: Keinmal oder einmal
* »`{m,n}`«: Mindestens m-mal, höchstens n-mal, z.B. »`{2,4}`«.

*Möglichst viel oder möglichst wenig?*

In zweideutigen Fällen kann angegeben werden, ob möglichst viele oder möglichst wenige Zeichen gefunden werden sollen.

* »`*?`« oder »`+?`«: Möglichst wenig

Für eine ausführliche Beschreibung sei auf [Wikipedia](http://de.wikipedia.org/wiki/Regul%C3%A4rer_Ausdruck#Regul.C3.A4re_Ausdr.C3.BCcke_in_der_Praxis) und die [Python-Dokumentation](https://docs.python.org/2.7/library/re.html) verwiesen.

Zur Veranschaulichung nun ein Beispiel: In einem Text sollen alle Passagen gefunden werden, die zwischen Anführungszeichen stehen:

In [6]:
text = u'Mehr als »Tür«: »Doppel-Tür« oder »Tür und Tor«.'

In [7]:
import re
re.findall(u'»\w+«', text, re.UNICODE)

[u'\xbbT\xfcr\xab']

In [8]:
re.findall(u'»[\w-]+«', text, re.U)

[u'\xbbT\xfcr\xab', u'\xbbDoppel-T\xfcr\xab']

In [9]:
re.findall(u'».+«', text, re.U)

[u'\xbbT\xfcr\xab: \xbbDoppel-T\xfcr\xab oder \xbbT\xfcr und Tor\xab']

In [10]:
re.findall(u'».+?«', text, re.U)

[u'\xbbT\xfcr\xab', u'\xbbDoppel-T\xfcr\xab', u'\xbbT\xfcr und Tor\xab']

In vielen Fällen soll aus einem Muster nur ein Element (oder mehrere Elemente) extrahiert werden. Dazu können Passagen im Muster eingeklammert und diese sogenannten »Gruppen« später isoliert werden.

In [11]:
text = u'Dieser Beitrag wurde veröffentlicht am 20.1.2014 von Marie Müller.'
pattern = u'veröffentlicht am ([\d\.]+) von (.*?)\.'
match = re.search(pattern, text, re.U)
match.group(1), match.group(2)

(u'20.1.2014', u'Marie M\xfcller')

Im Gegensatz zu `re.findall()` gibt `re.search()` die Treffer also nicht direkt zurück, sondern speichert das Ergebnis in einer Form, die den gezielten Zugriff auf verschiedene Bestandteile erlaubt. `re.search()` kann dabei auch als Test verwendet werden: Wenn das Muster nicht auf den Text passt, wird statt eines Match-Objekts der Wert `None` zurückgegeben.

### Aufgabe

Extrahieren Sie aus den gegebenen Daten ein Datumsobjekt (`datetime.date`). Ggf. müssen Sie dazu mit einer Fallunterscheidung arbeiten.

In [12]:
data = [
    u'Datum: 23.1.2011',
    u'Datum: 12. Januar 2004',
    u'Veröffentlicht am 5. September 1999.',
    u'Publiziert 12.04.2012 von admin',
]

In [13]:
def extract_date(text):
    return None

[extract_date(text) for text in data]

[None, None, None, None]

## Ein Testfall

Die von der Seite der Bundesregierung gespeicherten Reden enthalten zwar Angaben zu Ort und Zeit, aber die Redner sind nicht explizit ausgewiesen. Sie werden allerdings in der Regel im Titel genannt. Ein Ansatz kann also sein, Angaben zur Rednerin bzw. zum Redner mittels regulärer Ausdrücke aus dem Titel zu extrahieren.

In [14]:
import pandas as pd
data = pd.read_csv('../Daten/reden.csv', parse_dates=['date'], encoding='utf-8')

In [15]:
data.head()

,place,title,date,text,abstract,link
0,Weimar,Laudatio auf Rüdiger Safranski,2014-07-06,- Es gilt das gesprochene Wort. -\r\nEs gehört...,Der Schriftsteller und Literaturwissenschaftle...,http://www.bundesregierung.de/Content/DE/Rede/...
1,Peking,Rede von Bundeskanzlerin Merkel anlässlich der...,2014-07-08,"Sehr geehrter Herr Minister Miao Wei,\r\nmeine...",in Peking\r\n,http://www.bundesregierung.de/Content/DE/Rede/...
2,Peking,Rede von Bundeskanzlerin Merkel anlässlich des...,2014-07-08,"Sehr geehrter Herr Präsident,\r\nsehr geehrte,...",NaN,http://www.bundesregierung.de/Content/DE/Rede/...
3,Alliiertenmuseum Berlin,Kulturstaatsministerin Monika Grütters zur Erö...,2014-07-08,"- Es gilt das gesprochene Wort. -\r\nAnrede,\r...","""Wer verstehen will, warum uns mit den USA wei...",http://www.bundesregierung.de/Content/DE/Rede/...
4,NaN,Rede von Bundeskanzlerin Merkel anlässlich der...,2014-07-11,"Sehr geehrter Herr Meyer,\r\nsehr geehrte Frau...",in Rostock-Warnemünde\r\n,http://www.bundesregierung.de/Content/DE/Rede/...


In [16]:
titles = data['title']
titles.head()

0                       Laudatio auf Rüdiger Safranski
1    Rede von Bundeskanzlerin Merkel anlässlich der...
2    Rede von Bundeskanzlerin Merkel anlässlich des...
3    Kulturstaatsministerin Monika Grütters zur Erö...
4    Rede von Bundeskanzlerin Merkel anlässlich der...
Name: title, dtype: object

In [17]:
def extract_speaker(text):
    match = re.search(u'Rede von ([A-ZÄÖÜ]\w+(?:\s+[A-ZÄÖÜ]\w+)*)', text, re.U)
    if match:
        return match.group(1)
    else:
        return None
    
speakers = titles.apply(extract_speaker)  # entspricht [extract_speaker(title) for title in titles]
speakers.head()

0                      None
1    Bundeskanzlerin Merkel
2    Bundeskanzlerin Merkel
3                      None
4    Bundeskanzlerin Merkel
Name: title, dtype: object

In [18]:
speakers.value_counts()

Bundeskanzlerin Angela Merkel                  177
Bundeskanzlerin Merkel                         137
Kulturstaatsminister Bernd Neumann              89
Staatsminister Bernd Neumann                    30
Bundeskanzlerin Angela Merkel                   14
Kulturstaatsministerin Grütters                  3
Kulturstaatsministerin Monika Grütters           3
Kulturstaatsminister Bernd Neumann               2
Bundeskanzlerin Merkel                           2
Bundeslandwirtschaftsministerin Ilse Aigner      1
Bundesminister Pofalla                           1
Bundeskanzlerin  Angela Merkel                   1
Kulturstaatsministers Bernd Neumann              1
Joachim Gauck                                    1
Bundesaußenminister Steinmeier                   1
Kanzlerin Merkel                                 1
BK                                               1
Kulturstaatsinister Bernd Neumann                1
Staatsminister Bernd Neumann                     1
dtype: int64

In [19]:
len(speakers), sum(speakers.isnull())

(614, 147)